In [1]:
# script to extract data from g adventures API


In [2]:
# do i need the full name for itinerary name?

In [3]:
# get promotional rates including promotion e.g. 25% of

In [4]:
from gapipy import Client
from datetime import timedelta
import pandas as pd

In [5]:
import re
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("G_ADVENTURES_KEY")

client = Client(application_key=api_key) 

In [6]:
# filtered_departures = client.departures.filter(start_date="2024-02-19")

In [7]:
filtered_departures = client.departures.filter(start_date__gt="2024-04-01", finish_date__lt="2024-04-30")

In [8]:
filtered_departures.count()

3466

In [9]:
my_generator = filtered_departures.all()

In [10]:
departures_dictionaries_list = []
countries_set = set()
reference_count = 0
for value in my_generator:
    reference_count += 1
    if reference_count % 100 == 0:
        print(reference_count)
    try:
        if value.availability['status'] == "AVAILABLE" and value.tour_dossier != None: 
            trip_list = [category['name'] for category in value.tour_dossier.categories if category['category_type']['label'] ==  "Trip Type"]
            if 'Small Group' in trip_list:
                for i in range(len(value.structured_itineraries)):

                    # get tour name
                    tour_name = value.tour_dossier.name

                    # structured itinerary id
                    structured_itineraries_name = value.structured_itineraries[i].name

                    # get the list of countries visited
                    visited_countries_list = []
                    for country in value.tour_dossier.geography['visited_countries']:
                        visited_countries_list.append(country["name"])
                        countries_set.add(country["name"]) #  add countries to set

                    
                    # get start date
                    start_date = value.start_date
                
                    # get duration
                    duration = value.finish_date - value.start_date + timedelta(days=1)
                    duration = duration.days

                    # get pricing form room categories
                    room_categories_dictionaries = {"currency": "GBP"}
                    
                    for room in value.rooms:
                        data = room.to_dict()
                        for price_brand in data["price_bands"]:
                            for price in price_brand["prices"]:
                                if price['currency'] == "GBP":
                                    # get prices
                                    price_column = f'{data["name"]} - {price_brand["name"]}'
                                    if price_column in room_categories_dictionaries:
                                        room_categories_dictionaries[price_column] += price['amount']
                                    else:
                                        room_categories_dictionaries[price_column] = price['amount']
                                    # get promotion
                                    if  len(price['promotions']) > 0:
                                        print("started promo")
                                        # get promo cost
                                        promotion_column_cost = f'{data["name"]} - {price_brand["name"]} - Promotion cost'
                                        if promotion_column_cost in room_categories_dictionaries:
                                            room_categories_dictionaries[promotion_column_cost] += price["promotions"][0]['amount']
                                        else:
                                            room_categories_dictionaries[promotion_column_cost] = price["promotions"][0]['amount']
                                        # promo cost done
                                        # get promo description
                                        promotion_column_name = f'{data["name"]} - {price_brand["name"]} - Promotion description'
                                        if promotion_column_name in room_categories_dictionaries:
                                            room_categories_dictionaries[promotion_column_name] += client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                        else:
                                            room_categories_dictionaries[promotion_column_name] = client.promotions.get(price["promotions"][0]['id']).to_dict()['name']
                                        print("finished promo")
                                            
                    # get tour url 
                    tour_url = value.tour_dossier.site_links[0]['href']
                    
                    # get itinerary day by day
                    itinerary = ""
                    itinerary_days = value.structured_itineraries[i]
                    for day in itinerary_days.days:
                        itinerary += f'Day {day.day}: {day.summary}\n'
                    
                    # format itinerary text to add "" before and after
                    format_functions = lambda x: f'"{x}"'
                    itinerary = format_functions(itinerary)

                    # get tour categories
                    categories_id_list = [i['id'] for i in value.tour_dossier.categories]

                    tour_categories_dictionaries = {}

                    for category in categories_id_list:
                        category_dict = client.tour_categories.get(category).to_dict()
                        category_value = None
                        if category_dict['description'] == None:
                            category_value = category_dict['name']
                        else:
                            category_value = f"{category_dict['name']} - {category_dict['description']}"

                        if category_dict['category_type']['name'] in tour_categories_dictionaries:
                            tour_categories_dictionaries[category_dict['category_type']['name']] += ", " + category_value
                        else:
                            tour_categories_dictionaries[category_dict['category_type']['name']] = category_value

                    # create dictionary
                    dataframe_dictionary = {}
                    dataframe_dictionary['tour_operator'] = "G Adventures"
                    dataframe_dictionary['tour_name'] = tour_name
                    dataframe_dictionary['itinerary_name'] = structured_itineraries_name
                    dataframe_dictionary['visited_countries'] = ", ".join(visited_countries_list)
                    dataframe_dictionary['start_date'] = start_date
                    dataframe_dictionary['duration'] = duration
                    dataframe_dictionary['url'] = tour_url
                    dataframe_dictionary['itinerary'] = itinerary


                    # combine dictionaries
                    dataframe_dictionary = dict(dataframe_dictionary, **tour_categories_dictionaries, **room_categories_dictionaries)


                    
                    # add full dict to list
                    departures_dictionaries_list.append(dataframe_dictionary)
     
    except:
        print("403 issue")


403 issue
403 issue
403 issue
100
200
300
400
500
600
700
800
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
900
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
1000
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
1100
403 issue
403 issue
403 issue
403 issue
403 issue
403 issue
1200
1300
1400
403 issue

In [11]:
df = pd.DataFrame(departures_dictionaries_list)
df.to_csv('raw_data/one_month.csv', encoding='utf-8', index=False)
df.head(3)

,tour_operator,tour_name,itinerary_name,visited_countries,start_date,duration,url,itinerary,Travel Style,Service Level,...,Shared bathroom - Double/Twin - D - Adult,Shared bathroom - Mixed gender dorm - B - Adult,Standard - Double - D - Adult,Upper Deck - Twin - D - Adult,Standard - Child,Non View En suite bathroom - Twin - D - Adult,View En suite bathroom - Twin - D - Adult,Non View En suite bathroom - Double - D - Adult,Main Deck - Twin - D - Adult,Lower Deck - Twin - D - Adult
0,G Adventures,Sailing Thailand - Ko Phi Phi to Phuket,,Thailand,2024-04-02,4,https://www.gadventures.com/trips/sailing-thai...,"""Day 1: Meet in Ko Phi Phi in the early aftern...",Marine - Small-ship adventures on the world’s ...,Standard - Comfortable tourist-class accommoda...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G Adventures,Sailing Thailand - Phuket to Phuket,,Thailand,2024-04-06,7,https://www.gadventures.com/trips/sailing-thai...,"""Day 1: Meet in Phuket in the early afternoon....",Marine - Small-ship adventures on the world’s ...,Standard - Comfortable tourist-class accommoda...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G Adventures,Sailing Thailand - Phuket to Ko Phi Phi,,Thailand,2024-04-06,4,https://www.gadventures.com/trips/sailing-thai...,"""Day 1: Meet in Phuket in the early afternoon....",Marine - Small-ship adventures on the world’s ...,Standard - Comfortable tourist-class accommoda...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
list(countries_set)

['Nicaragua',
 'Jordan',
 'New Zealand',
 'Colombia',
 'Maldives',
 'Uzbekistan',
 'Monaco',
 'Italy',
 'Austria',
 'Iceland',
 'Argentina',
 'Oman',
 'Malaysia',
 'Poland',
 'Bosnia and Herzegovina',
 'Laos',
 'Hungary',
 'Zambia',
 'Bhutan',
 'United States',
 'Turkey',
 'Morocco',
 'Montenegro',
 'Serbia',
 'Belgium',
 'China',
 'Peru',
 'Israel',
 'Egypt',
 'Netherlands',
 'Indonesia',
 'Sri Lanka',
 'Czech Republic',
 'Honduras',
 'Mexico',
 'Madagascar',
 'Greece',
 'Cuba',
 'Kenya',
 'Turkmenistan',
 'Belize',
 'Portugal',
 'Cambodia',
 'Rwanda',
 'Botswana',
 'Spain',
 'India',
 'Switzerland',
 'Australia',
 'Uruguay',
 'Uganda',
 'Namibia',
 'Croatia',
 'France',
 'South Korea',
 'Brazil',
 'Zimbabwe',
 'Chile',
 'Thailand',
 'South Africa',
 'El Salvador',
 'Nepal',
 'Japan',
 'Guatemala',
 'Bolivia',
 'Slovenia',
 'Germany',
 'Ecuador',
 'Singapore',
 'Vietnam',
 'Costa Rica',
 'Tanzania',
 'Malawi']